Notebook creado por [Derrick Schultz](https://twitter.com/dvsch) and [Lia Coleman](https://twitter.com/Lialialiacole).

Traducido por @hypereikon

In [ ]:
#@markdown setear tensorflow 1
%tensorflow_version 1.x

In [ ]:
#@markdown Podemos añadir funciones para visualizar. Esta celda permitirá mostrar imagenes y videos directamente en Colab.
from IPython.display import Image, display, HTML
import base64
import io

def show_local_mp4_video(file_name, width=640, height=480):
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))
def make_img_grid(images,width=360):
    html = []
    for image in images:
        with open(image, "rb") as img_file:
            my_string = base64.b64encode(img_file.read())
            img_uri = "data:image/png;base64," + my_string.decode('utf8')
        html.append('<img src="{}" style="width:{}px;display:inline;margin:1px"/>'.format(img_uri,str(width)))
    return ''.join(html)
    

In [ ]:
#@markdown Instalar el repositorio de StyleGAN2-ADA 
!git clone https://github.com/dvschultz/stylegan2-ada
!pip install opensimplex
%cd /content/stylegan2-ada

##Descargar un modelo StyleGAN2-ADA
Modelos:

Wikiart
https://archive.org/download/wikiart-stylegan2-conditional-model/WikiArt_Uncond2.pkl

Metfaces
https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/metfaces.pkl

FFHQ
https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

In [ ]:
!wget https://archive.org/download/wikiart-stylegan2-conditional-model/WikiArt_Uncond2.pkl -O /content/network.pkl

In [ ]:
#!gdown --id 10e6uRa9Qxx_pano8QGVXrStqM-r3dcEd -O /content/network.pkl

## Generacion

Tambien se le llama inferencia, evaluación o testeo del modelo.

In [ ]:
#@markdown detalle parametros
!python generate.py --help

####Generacion de imagenes

`--network`: Asegurate que el argumento de `--network` apunte a tu archivo .pkl . 
(El metodo más utilizado es hacer click derecho en el archivo del panel de archivos y seleccionar `Copy Path`, luego pegar esto en el argumento despues del signo `=`)

`--seeds`: Esto te permite elegir un seed random desde el modelo. Recuerda que el input para StyleGAN es un arreglo 512-dimensional. Estos seeds generaran estos 512 valores. Cada seed generará un diferente arreglo random. El mismo valor del seed siempre generará el mismo arreglo random, entonces siempre podremos usarlo para otros procesos como interpolación. 

`--trunc`: Esto setea el monto de truncation.

`--outdir`: Donde guardar los resultados. 

`--dlatents=` Render image from projected latent vector `--dlatents=out/dlatents.npz`

In [ ]:
!python generate.py generate-images --network="/content/network.pkl" --seeds=0,1,2,3,5,8,13,21 --outdir="./out/" --trunc=0.5

Ahora miremos unas cuantas imagenes, asegurate que sean las mismas seed generadas anteriormente.

In [ ]:
listOfImageNames = ['/content/stylegan2-ada/out/seed0000.png',
                    '/content/stylegan2-ada/out/seed0001.png',
                    '/content/stylegan2-ada/out/seed0002.png',
                    '/content/stylegan2-ada/out/seed0003.png',
                    '/content/stylegan2-ada/out/seed0005.png',
                    '/content/stylegan2-ada/out/seed0008.png',
                    '/content/stylegan2-ada/out/seed0013.png',
                    '/content/stylegan2-ada/out/seed0021.png',          ]
display(HTML(make_img_grid(listOfImageNames, 200)))


In [ ]:
#@markdown Comprimamos y descarguemos las imagenes generadas.
!zip -r generated-out.zip /content/stylegan2-ada/out

Nota: Los comandos anteriores guardaran las imagenes en la carpeta llamada `out`, si ya generaste y tienes cosas en la carpeta `out` asegurate de eliminarlo porque se incluirá en el zip. 

Para eliminar toda una carpeta desde colab, dale click a la siguiente celda: 

In [ ]:
!rm -r "./out/"

#### Truncation transversal
Truncation, trunca el espacio latente. Esto puede tener un sutil o dramatico efecto en las imagenes dependiendo del valor que uses. Mayormente se utiliza entre 0.5 y 1.0 para resultados coherentes con tu dataset, pero tecnicamente esto puede llegar hasta el infinito (+ y -). 

Abajo tu puedes tomar un seed y mirar como varia según el valor del truncation. -1 a 1 serán imagenes bastante realistas, pero alejandose de estos se vuelve más extraño. 

Opciones
`--network`: La ruta de tu archivo .pkl

`--seed`: Pasa solo una seed. Selecciona tu favorita de las generadas anteriormente. 

`--start`: Valor de truncation inicial.

`--stop`: Valor de truncation final. Deberia ser diferente al inicial. Probablemente colapse sino.

`--increment`: Que tanto debería incrementar el trucation entre cada frame. Asegurate que sea un valor bien pequeño si quieres una larga y lenta interpolación.

(stop-start/increment=frames total)

In [ ]:
!python generate.py truncation-traversal --network="/content/network.pkl" --seed=13 --start=-0.3 --stop=1.5 --increment=0.05 --outdir="./tt" --fps=30

Ahora debería haber un video llamado: `truncation-traversal-seed-start-stop.mp4` en la carpeta `tt`. Puedes descargarlo con el click derecho.

Si quieres visualizarlo desde colab, actualiza la ruta y ejecuta la siguiente celda.  

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/tt/truncation-traversal-seed13-start-0.3-stop1.5.mp4", width=512, height=512)

### Interpolaciones
Interpolacion es el proceso de aplicarle pequeños cambios a un vector, pareciendo asi una animacion de frame en frame.

Veremos dos diferentes formas de interpolar: interpolacion lineal y random noise loop.

Ambos metodos requieren las siguientes opciones:

`--network`

`--walk-type`: Walk type define el tipo de interpolacion a usar. En algunos casos tambien puedes definir interpolar en el espacio latente z o w.

`--frames`: Cuantos frames producir en total. Esto determinara el largo del video.

`--trunc`: truncation value


#### Interpolacion lineal en el espacio z

Interpolacion lineal genera una linea entre una seed y otra. Los creadores de Stylegan dicen que esto produce resultados más *desenredados* (disentangled) en el espacio w. Pero empecemos mirandolo en el espacio z.

`--seeds`: Usa las imagenes creadas previamente para controlar los puntos de interpolacion. Si tu primera y ultima seed son la misma producira un loop (ideal para instagram y gifs!)

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="line-z" --seeds=0,1,2,3,8,13,21,0 --outdir="./z-walk" --frames 600


Si quieres visualizarlo desde colab, actualiza la ruta y ejecuta la siguiente celda.  

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/z-walk/---.mp4", width=512, height=512)

#### Interpolacion lineal en el espacio w
Ahora miremos la interpolacion lineal en el espacio w. Oara esto seteamos `--walk-type` a `line-w`.

Recomendamos usar las mismas seeds del anterior para ver la diferencia entre ambas. Es bien sutil pero es diferente.

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="line-w" --seeds=0,1,2,3,8,13,21,0 --outdir="./w-walk" --frames 600  --trunc=0.95


Si quieres visualizarlo desde colab, actualiza la ruta y ejecuta la siguiente celda.  

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/w-walk/walk-w-line0-1-2-3-8-13-21-0-24fps.mp4", width=512, height=512)

#### Interpolacion de Noise Loop

Si quieres crear interpolaciones random de tu modelo, noise loop es el ideal. Crea una linea random a traves del espacio latente z, generando asi imagenes diversas. It creates a random path thru the z space to show you a diverse set of images.

`--diameter`: Esto controla que tan "ancho" el loop será. El hacerlo más pequeño genera un set menos diverso de samples. Hacerlo más largo generar un set más diverso de samples, así mismo más cantidad de samples. Esto junto a `--frames` puede ayudar a determinar que tan rapido se siente el video. 

`--start_seed`: Esto permite cambiar el punto de inicio en el espacio z. Nota: esta seed no tiene relacion con las seeds usadas anteriormente para generar imagenes. Solo permite que cambies el punto del centro del loop (asi puedes regresar a este usando denuevo esta seed).

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="noiseloop" --start_seed=999 --outdir="./noise1" --diameter=8.0 --frames=500

Si quieres visualizarlo desde colab, actualiza la ruta y ejecuta la siguiente celda.

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/noise1/walk-z-noiseloop-seed999-24fps.mp4", width=512, height=512)

### Style Mixing, Mezcla de estilo

Ya que las seeds son solo puntos en un espacio vectorial multidimensional, podemos aplicarle operaciones matematicas a estos, como por ejemplo sumarlos entre si. Esto se podria hacer mediante interpolacion, usando el frame de al medio, pero si quieres ver las diferentes opciones aqui hay un script simple para hacerlo. Esto toma un numero de seeds para producir una grilla mostrando que pasa cuando se suman las filas y columnas entre si (esto tendra más sentido despues de ejecutarlo).

In [ ]:
!python style_mixing.py --outdir="./stylemix" --rows=0,1,2,3 --cols=5,8,13,21 --network="/content/network.pkl"

### Proyeccion

Proyeccion es el proceso de tomar una imagen de fuera del modelo (no generada ni usada en el dataset original) y encontrar su mas parecida representacion dentro del modelo.

Primero, creemos unas carpetas donde subiremos las imagenes.

In [ ]:
!mkdir uploads; mkdir aligned

Ahora, sube una imagen a la carpeta `uploads`. Recomiendo encontrar un retrato a una resoucion decente (~1500px)

Luego necesitamos asegurarnos que el retrato este alineado similar al dataset, ejecuta la siguiente celda para alinear la imagen.

***solo si estas usando un dataset de caras de personas.***

In [ ]:
!python align_faces.py ./uploads ./aligned

Ahora estamos listos para procesar la proyeccion.

#### Opciones 
`--target`: sube una imagen que quieras proyectar dentro del modelo, luego setea la opcion `--target` con su ruta. Esta puede ser cualquier imagen, mientras mantenga la misma relacion de aspecto con dataset que fue entrenado el modelo.

`--network`: la ruta de tu modelo. 


In [ ]:
!python projector.py --outdir="./projection" --target=/content/stylegan2-ada/aligned/br_01.png --network="/content/network.pkl"

Si quieres visualizarlo desde colab, actualiza la ruta y ejecuta la siguiente celda.

In [ ]:
show_local_mp4_video("/content/stylegan2-ada/projection/proj.mp4", width=1024, height=512)

## Diversion: Flesh Digressions

[Flesh Digressions](https://aydao.ai/artwork/2020/01/17/fleshdigressions.html) es una tecnica desarrollada por [aydao](https://twitter.com/aydaogman). Este comando generara un video flesh digressions llamado `circular-25-10-2020-09-00-34-PM.mp4`.

In [ ]:
!python aydao_flesh_digressions.py --pkl "/content/network.pkl" --psi=0.6 --radius_small=40.0 --radius_large=300.00

Si quieres visualizarlo desde colab, actualiza la ruta y ejecuta la siguiente celda.

In [ ]:
show_local_mp4_video("./circular-27-10-2020-10-52-03-PM.mp4", width=512, height=512)